In [61]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from scipy.sparse import hstack, vstack
import csv
import re
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import gensim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
from sklearn.ensemble import RandomForestRegressor
import nltk
import string
import contractions
nltk.download('wordnet')

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^a-z ]')
STOPWORDS = set(stopwords.words('english'))
stemmer = nltk.stem.WordNetLemmatizer()

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = " ".join([contractions.fix(t) for t in text.split(' ')])
    text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE, '', text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word.strip(string.punctuation) for word in text.split(' ')])
    text = ' '.join([t for t in text.split(" ") if len(t) > 2])
    text = ' '.join([stemmer.lemmatize(x) for x in text.split() if x not in STOPWORDS])# delete stopwords from text
    return text


[nltk_data] Downloading package wordnet to /home/bhawesh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
goog_embed = "/usr/workspace/tutorial/DL/Natural-Language-Processing/week3/data/GoogleNews-vectors-negative300.bin"
wv_embeddings = KeyedVectors.load_word2vec_format(datapath(goog_embed), binary=True, limit=500000)
def question_to_vec(question, embeddings, dim=300):
    """
        question: a string
        embeddings: dict where the key is a word and a value is its' embedding
        dim: size of the representation

        result: vector representation for the question
    """
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    count = 0
    vec = np.zeros(dim)
    for word in question.split(" "):
        if word in embeddings:
            count += 1
            vec += embeddings[word]
    if count == 0:
        return vec
    return vec/count
            
    

/home/bhawesh/.local/lib/python3.5/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [62]:
# 0-positive, 1-negative, 2-neutral
df = pd.read_csv("inno/train.csv")
# print(df["text"].iloc[0])
df["text"] = df["text"].apply(text_prepare)
# print(df["text"])
x = df[["text", "drug"]]
train_y = df["sentiment"]
train_x = df["text"] + ' ' + df["drug"]

In [34]:
# word2vec approach
question2vec_result = []
for con in range(len(train_x)):
#     question = question.strip()
    answer = question_to_vec(train_x.iloc[con], wv_embeddings)
    question2vec_result.append(answer)
    
# tfidf = TfidfVectorizer(ngram_range=(1,2))
# features = tfidf.fit_transform(np.array(train_x)) #fit transform to learn vocubulary and transform in matrix

tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1,3))
features1 = tfidf.fit_transform(np.array(x["text"])) #fit transform to learn vocubulary and transform in matrix
# features_test = tfidf.transform(np.array(testing_phrase["text"])) #uses vocab from fit and transform to matrix
features2 = tfidf.transform(np.array(x["drug"]))
features = hstack([features1, features2])

regressor = RandomForestRegressor(n_jobs=-1, n_estimators=100)

# regressor.fit(features, np.array(train_y))
regressor.fit(np.array(question2vec_result), np.array(train_y))

In [64]:
# tfidf = TfidfVectorizer(ngram_range=(1,2))
# features = tfidf.fit_transform(np.array(train_x)) #fit transform to learn vocubulary and transform in matrix

tfidf = TfidfVectorizer(min_df=5, max_df=0.5, ngram_range=(1,3))
features1 = tfidf.fit_transform(np.array(x["text"])) #fit transform to learn vocubulary and transform in matrix
# features_test = tfidf.transform(np.array(testing_phrase["text"])) #uses vocab from fit and transform to matrix
features2 = tfidf.transform(np.array(x["drug"]))
features = hstack([features1, features2])

regressor = RandomForestRegressor(n_jobs=-1, n_estimators=100, verbose=2)

regressor.fit(features, np.array(train_y))
# regressor.fit(np.array(question2vec_result), np.array(train_y))

building tree 1 of 100building tree 2 of 100building tree 3 of 100building tree 4 of 100



building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  2.6min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  7.8min finished


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
           oob_score=False, random_state=None, verbose=2, warm_start=False)

In [65]:
# tfidf approach
df1 = pd.read_csv("inno/test.csv")
df1["text"] = df1["text"].apply(text_prepare)
x1 = df1["text"] + ' ' + df1["drug"]
y1 = df["sentiment"]

# feature1 = tfidf.transform(x1)

features3 = tfidf.transform(np.array(df1["text"]))
features4 = tfidf.transform(np.array(df1["drug"]))
features11 = hstack([features3, features4])

x3 = regressor.predict(features11)

x4 = pd.DataFrame({'unique_hash':np.array(df1["unique_hash"]), 'sentiment':x3})
x4["sentiment"] = x4["sentiment"].apply(lambda x:int(round(x)))
x4.to_csv("inno/sub15.csv", index=False)

[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.2s finished


In [36]:
# word2vec approach
df1 = pd.read_csv("inno/test.csv")
df1["text"] = df1["text"].apply(text_prepare)
x1 = df1["text"] + ' ' + df1["drug"]
y1 = df["sentiment"]

# feature1 = tfidf.transform(x1)

# features3 = tfidf.transform(np.array(df1["text"]))
# features4 = tfidf.transform(np.array(df1["drug"]))
# features11 = hstack([features3, features4])

question2vec_result1 = []
for con1 in range(len(x1)):
#     question = question.strip()
    answer1 = question_to_vec(x1.iloc[con1], wv_embeddings)
    question2vec_result1.append(answer1)

x3 = regressor.predict(np.array(question2vec_result1))

x4 = pd.DataFrame({'unique_hash':np.array(df1["unique_hash"]), 'sentiment':x3})
x4["sentiment"] = x4["sentiment"].apply(lambda x:int(round(x)))
x4.to_csv("inno/sub12.csv", index=False)

In [29]:
testit = []
testit.append(question_to_vec("hello how are you", wv_embeddings))
testit.append(question_to_vec("hello how", wv_embeddings))

In [32]:
np.array(testit).shape

(2, 300)

In [63]:
x

,text,drug
0,autoimmune disease tend come cluster gilenya f...,gilenya
1,completely understand would want try result re...,gilenya
2,interesting target receptor rather like fingol...,fingolimod
3,interesting grand merci wonder lemtrada ocrevu...,ocrevus
4,everybody latest mri result brain cervical cor...,gilenya
5,give advice lemtrada chose cladribine thought ...,cladribine
6,reply posted jesszidek jess sorry read challen...,humira
7,well expected neurologist want start tysabri k...,gilenya
8,think fingolimod miserable failure progressive...,fingolimod
9,thank much learning lot grace mentioned husban...,tagrisso
